# 第二章 关键词检索


在本章中，您将学习如何使用关键词搜索（Keyword Search）并利用数据库回答问题。搜索对于我们在世界中导航非常关键。它包括搜索引擎，也包括在应用程序内部进行搜索，比如在 Spotify、YouTube 或 Google 地图中进行搜索。公司和组织也需要使用关键词检索或其他各种搜索方法来搜索其内部文件。关键词检索是构建搜索系统最常用的方法。接下来，让我们看看如何使用关键词检索系统，然后再看看语言模型如何改进这些系统。

在本章教程中，我们需要用到 Weaviate 和 Cohere 的 API key。

## 目录

- [一、环境配置](#一、)

- [二、Weaviate 数据库](#二、)

  - [2.1 进行身份验证配置](#2.1)
  
  - [2.2 连接数据库](#2.2)
  
- [三、关键词检索](#三、)

  - [3.1 构建关键词检索函数](#3.1)
  
  - [3.2 使用关键词检索函数](#3.2)
  
- [四、关键词检索的更深理解](#四、)

- [五、关键词检索的限制](#五、)

## 一、环境配置  <a id="一、"></a>

让我们先准备好需要用到的一些 Python 库和 API：

In [ ]:
!pip install cohere
!pip install weaviate-client
!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # 读取本地 .env 文件

## 二、Weaviate 数据库  <a id="二、"></a>

Weaviate 是一个开源数据库。它具有关键字检索功能，也具有依赖于语言模型的向量搜索功能。

### 2.1 进行身份验证配置  <a id="2.1"></a>

In [ ]:
import weaviate
auth_config = weaviate.auth.AuthApiKey(
    api_key=os.environ['WEAVIATE_API_KEY'])  # 获取环境变量中的 Weaviate API密钥，进行身份验证。

现在，我们已经设置了身份验证的配置，让我们看一下如何将客户端连接到实际数据库。

### 2.2 连接数据库  <a id="2.2"></a>

+ weaviate.Client() ：Weaviate客户端对象。

+ url ：Weaviate客户端的URL属性。这个URL指定了与Weaviate服务进行通信的位置。

+ auth_client_secret ：Weaviate客户端的身份验证密钥属性

+ additional_headers ：额外的请求头信息。

In [ ]:
client = weaviate.Client(
    url=os.environ['WEAVIATE_API_URL'],  
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": os.environ['COHERE_API_KEY'],  # 这里添加了一个名为 X-Cohere-Api-Key 的请求头，其值为环境变量中的 Cohere API密钥。
    }
)

这个数据库是一个公共数据库，包含了 1000 万条记录。这些记录来自维基百科。每个单元格、每个记录、每一行都是来自维基百科的一个段落。
这 1000 万条记录来自 10 种不同的语言。其中 100 万条是英文的，其余 900 万条是其他语言的。我们可以选择和过滤我们想要查询的语言，这个我们稍后会学到。

运行下面这行代码后，我们确保客户端已经准备好并连接上了。如果返回 True，那就意味着我们的本地 Weaviate 客户端能够连接到远程 Weaviate 数据库。然后我们就能在这个数据集上进行关键词检索了。

In [ ]:
client.is_ready() 

## 三、关键词检索  <a id="三、"></a>

让我们先简单了解一下关键词检索。

假设您有一个查询：“草是什么颜色？”，并且您在一个非常小的文档集中进行搜索，其中包含了以下五个句子：“明天是星期六”，“草是绿色的”，“加拿大的首都是渥太华”，“天空是蓝色的”，“鲸鱼是哺乳动物”。

这是一个简单的搜索示例。关键词搜索的工作原理是比较**查询**和**文档**之间有多少共同的单词。如果我们比较查询和第一句话之间有多少共同的单词，我们可以发现它们只有一个共同的词：“是”。
我们可以统计这个文档集中每个句子的词语计数情况。然后我们可以发现第二个句子与查询有最多的共同词，因此关键词搜索可能会将其作为答案返回。

下面我们开始学习如何使用关键字检索。


### 3.1 构建关键词搜索函数  <a id="3.1"></a>

在之前的代码中我们已经连接到数据库，现在让我们构建一个查询数据库的函数。我们将其称为"keyword_search"。

我们先输入"respnse=()"，然后在里面设置我们需要的参数。

我们需要明确添加的集合的类型，这里需要添加的集合是 Articles（文章），它是这个数据库中已经定义好的类型，并且该数据库中的每篇 Articles 都有许多属性。

对于这个搜索，我们希望输入一个查询时，能够返回每个结果的 title（标题）、url、和正文（text）。这里还有许多其他属性，但此时我们不需要数据库将其返回。因此我们写入"client.query.get("Articles", properties)"构建一个查询请求，指定要查询的资源类型为 Articles ，并指定要返回的属性列表为 properties 参数的值，列表里面包含了属性 title , url , text 。

接下来处理关键词搜索部分，我们使用".with_bm25(query=query)"将关键词查询 query 添加到查询请求中。这将使用 BM25 算法对查询进行加权，以提高搜索结果的相关性。BM25 是一种常用的关键词检索或词汇搜索算法，它根据一个特定的公式对存档中的文档与查询进行评分，该公式考虑了查询与每个文档之间共享单词的数量。

然后我们使用".with_where(where_filter)"将语言过滤器添加到查询请求中，以限制搜索结果的语言，这里我们限制为中文。

最后使用".with_limit(num_results)"将结果数量限制添加到查询请求中，以指定要返回的搜索结果数量。这里我们将"num_results"设置为3，即我们默认返回3条结果。

有了这些，我们的查询就完成了。我们使用".do()"执行查询请求，并将结果存储在 response 变量中。现在函数就可以获取响应并返回结果了。

这就是我们的关键词检索函数。

In [2]:
def keyword_search(query,
                   results_lang='zh',
                   properties=["title", "url", "text"],
                   num_results=3):
    """
    关键词搜索函数

    参数：
    query：要搜索的关键词
    results_lang：搜索结果的语言，默认为中文（'zh'）
    properties：要返回的属性列表，默认为标题、URL和文本
    num_results：要返回的结果数量，默认为3个

    返回：
    搜索结果列表
    """

    # 构建过滤器，限制搜索结果的语言
    where_filter = {
        "path": ["lang"],
        "operator": "Equal",
        "valueString": results_lang
    }

    # 发送查询请求，获取搜索结果
    response = (
        client.query.get("Articles", properties)
        .with_bm25(
            query=query
        )
        .with_where(where_filter)
        .with_limit(num_results)
        .do()
    )

    # 提取搜索结果
    result = response['data']['Get']['Articles']
    return result

### 3.2 使用关键词检索函数  <a id="3.2"></a>

现在让我们使用这个关键词检索函数，并传递一个查询给它。
假设我们想搜索“中国”我们将查询传递给函数，然后将其打印出来，看看运行后会返回什么。

In [ ]:
query = "中国"
keyword_search_results = keyword_search(query)
print(keyword_search_results)

[{'text': '古时“中国”含义不一：或指天子所在的京师为“中国”。《》：“惠此中国，以绥四方。”毛传：“中国，京师也。”《》：“夫而后之中国，践天子位焉。”《集解》：“刘熙曰；‘帝王所都为中，故曰中国’。”或指华夏族、汉族地区为中国（以其在四夷之中）。《》：“《小雅》尽废，则四夷交侵，中国微矣。”又《》：“是以声名洋溢乎中国，施及蛮貊。”而华夏族多建都于黄河南、北，因称其地为“中国”，与“中土”、“中原”、“中州”、“中夏”、“中华”含义相同，初时本指今河南省北部、山西省南部和陕西省南部及附近地区，后来中原王朝的活动范围扩大，黄河中下游一带，也被称为“中国”。或指统辖中原之国，《》：“盂达于是连吴固蜀，潜图中国。”，也把所统辖的地区，包括不属于黄河流域的地方，也全部称为“中国”。《》：“其后秦遂以兵灭六国，并中国。”在统一的情况下，中央王朝常自称为“中国”；而分裂时期，“中国”也能指稱黄河中下游地区（即中原）或延續正統的王朝。《晉書·載記第十四》苻堅對其弟苻融言“劉禪可非漢之遺祚；然終為中國之所並”。此處「中国」指三國時期于華北地區的魏国，原因是魏繼承漢的正統。此外，古時「中國」一詞也具有單獨代指漢民族的用法。<br>', 'title': '中國的稱號', 'url': 'https://zh.wikipedia.org/wiki?curid=527278'}, {'text': '瑞士银行（中国）有限公司为瑞银子公司，其前身是2004年成立的瑞士银行有限公司北京分行。2012年3月，中国银监会发布《中国银监会关于由瑞士银行有限公司在中国境内分支机构改制的瑞士银行（中国）有限公司开业的批复》，批准瑞士银行（中国）有限公司，英文名为UBS (China) Limited 作为由瑞银单独出资的外商独资银行开业，北京市西城区金融大街7号英蓝国际金融中心1217－1230单元为注册营业地址；注册资本为20亿元人民币，近85%由瑞士银行有限公司拨付，其余部分由原瑞士银行有限公司在中国境内分行的营运资金划转；核准PETER ERIC WALSHE瑞士银行（中国）有限公司董事长的任职资格、金纪湘（SIMON JIXIANG JIN）瑞士银行（中国）有限公司行长的任职资格。允许其经营对各类客户的外汇业务及对除中国境内公民以外客户的人民币业务。 2012年7月，位于北京西城区的瑞士银行（中国）有限公司正式开业。', 'title': '瑞银集团', 'url': 'https://zh.wikipedia.org/wiki?curid=556866'}, {'text': '独特的外型更引起了时尚界的注意，使得她在Puma的Suede 50活动，Levi’s的新年TVC，以及路易威登2018年的展览和活动中悉数登场；并出现在多个亚洲尖端时尚生活杂志的版面，包括Vogue me（中国）、时尚芭莎（中国）、Nylon（中国/日本）、Ellemen睿士、大都市Numéro（中国）、红秀GRAZIA。（凤凰网音乐）', 'title': '刘柏辛', 'url': 'https://zh.wikipedia.org/wiki?curid=6070776'}]

这就是我们得到的搜索结果。这是一大段文本，但可以看到它是一个字典列表。所以让我们定义一个函数，以更好的方式打印它。

In [ ]:
def print_result(result):
    for i,item in enumerate(result):
        print(f'item {i}')
        for key in item.keys():
            print(f"{key}:{item.get(key)}")
            print()
        print()

调用这个函数，让我们看看清晰的结果是什么。

In [ ]:
print_result(keyword_search_results)

item 0
text:古时“中国”含义不一：或指天子所在的京师为“中国”。《》：“惠此中国，以绥四方。”毛传：“中国，京师也。”《》：“夫而后之中国，践天子位焉。”《集解》：“刘熙曰；‘帝王所都为中，故曰中国’。”或指华夏族、汉族地区为中国（以其在四夷之中）。《》：“《小雅》尽废，则四夷交侵，中国微矣。”又《》：“是以声名洋溢乎中国，施及蛮貊。”而华夏族多建都于黄河南、北，因称其地为“中国”，与“中土”、“中原”、“中州”、“中夏”、“中华”含义相同，初时本指今河南省北部、山西省南部和陕西省南部及附近地区，后来中原王朝的活动范围扩大，黄河中下游一带，也被称为“中国”。或指统辖中原之国，《》：“盂达于是连吴固蜀，潜图中国。”，也把所统辖的地区，包括不属于黄河流域的地方，也全部称为“中国”。《》：“其后秦遂以兵灭六国，并中国。”在统一的情况下，中央王朝常自称为“中国”；而分裂时期，“中国”也能指稱黄河中下游地区（即中原）或延續正統的王朝。《晉書·載記第十四》苻堅對其弟苻融言“劉禪可非漢之遺祚；然終為中國之所並”。此處「中国」指三國時期于華北地區的魏国，原因是魏繼承漢的正統。此外，古時「中國」一詞也具有單獨代指漢民族的用法。<br>

title:中國的稱號

url:https://zh.wikipedia.org/wiki?curid=527278


item 1
text:瑞士银行（中国）有限公司为瑞银子公司，其前身是2004年成立的瑞士银行有限公司北京分行。2012年3月，中国银监会发布《中国银监会关于由瑞士银行有限公司在中国境内分支机构改制的瑞士银行（中国）有限公司开业的批复》，批准瑞士银行（中国）有限公司，英文名为UBS (China) Limited 作为由瑞银单独出资的外商独资银行开业，北京市西城区金融大街7号英蓝国际金融中心1217－1230单元为注册营业地址；注册资本为20亿元人民币，近85%由瑞士银行有限公司拨付，其余部分由原瑞士银行有限公司在中国境内分行的营运资金划转；核准PETER ERIC WALSHE瑞士银行（中国）有限公司董事长的任职资格、金纪湘（SIMON JIXIANG JIN）瑞士银行（中国）有限公司行长的任职资格。允许其经营对各类客户的外汇业务及对除中国境内公民以外客户的人民币业务。 2012年7月，位于北京西城区的瑞士银行（中国）有限公司正式开业。

title:瑞银集团

url:https://zh.wikipedia.org/wiki?curid=556866


item 2
text:独特的外型更引起了时尚界的注意，使得她在Puma的Suede 50活动，Levi’s的新年TVC，以及路易威登2018年的展览和活动中悉数登场；并出现在多个亚洲尖端时尚生活杂志的版面，包括Vogue me（中国）、时尚芭莎（中国）、Nylon（中国/日本）、Ellemen睿士、大都市Numéro（中国）、红秀GRAZIA。（凤凰网音乐）

title:刘柏辛

url:https://zh.wikipedia.org/wiki?curid=6070776


我们总共得到了三个结果，每一个结果都是一段文本。由正文（text）、标题（title）、地址（url）组成。我们试图寻找的是跟中国有关的信息，可以看到三个结果里都多次出现了“中国”这个关键词。我们还可以看到每篇文章的 url，单击它，将进入该篇文章的维基百科页面。

您可以尝试修改查询，以查看数据集中还有什么内容。
在这里，你还可以尝试查看属性。下面是构建该数据集时使用的属性列表，这些属性都存储在数据库中

In [ ]:
properties = ["text", "title", "url", "views", "lang"]
# 其他可以尝试的语言：en, de, fr, es, it, ja, ar, zh, ko, hi

您可以通过查看属性 views 来查看维基百科页面收到的观看次数，并且用它来进行筛选或排序。

您还可以使用其他语言来进行筛选。可以尝试的其他语言有：英语、德语、法语、西班牙语、意大利语、日语、阿拉伯语、中文、韩语和印地语。只需输入其中一种语言，并将其传递给关键词检索，它将以该语言提供结果。不过在选择语言时，要注意所选语言的文档与查询必须要有共享的关键词。

BM25 只需要共享一个词，就可以将其评分为某种程度上相关。而且查询和文档共享的词越多，文档中重复的次数越多，得分就越高。

总的来说，关键词检索是一种可以展示查询与数据库的连接以及查询结果的高阶搜索。

## 四、关键词检索的更深理解  <a id="四、"></a>

接下来让我们从更高的层次上回顾一下**搜索**。

搜索的主要组成部分包括**查询**、**搜索系统**和**搜索系统可以访问的之前处理过的数据库**。搜索系统会根据与查询最相关的顺序给出一系列结果作为响应。

![Alt text](images/2-1.png)

如果我们更细致的看，可以将搜索系统视为具有多个阶段。第一阶段通常是检索或搜索阶段，之后还有一个称为重新排序（Reranking）的阶段。重新排序通常是必需的，因为我们希望包含或引入除了文本相关性之外的其他信息。

第一阶段的检索通常使用 BM25 算法来对数据库中的文档与查询进行评分。第一阶段检索的实现通常包含倒排索引（Inverted index）的概念。倒排索引是一种具有两列的表格。一列是关键词，旁边是包含该关键词的文档。这样做是为了优化搜索的速度。当你在搜索引擎中输入查询时，你肯定希望在几毫秒内就能得到结果。实际上，除了文档 ID 之外，关键词出现的频率也被添加到这个调用中。

![Alt text](images/2-2.png)

现在，请注意这个查询"What color is the sky?"，当我们查看倒排索引时，单词"color"对应文档804，而单词"sky"也对应文档804。因此，804将在第一阶段检索的结果中获得很高的评分。


## 五、关键词检索的限制  <a id="五、"></a>

通过我们对关键词检索的理解，我们可以看到它有一些限制。假设我们查询"Strong pain in the side ofthe head"，如果我们搜索到一个文档，这个文档中有另一个文档可以准确地回答它，比如"Sharp temple headache"，但它使用了不同的关键词，关键词搜索无法检索到这个文档。而这是语言模型可以帮助的一个领域，因为它们不仅仅比较关键词。还可以考虑到文档的含义，能够为查询检索到这样的文档。

![Alt text](images/2-3.png)

语言模型可以改进搜索的两个阶段，在接下来的课程中，我们将学习如何做到这一点。我们将看到语言模型如何通过 Embedding（嵌入）来改进检索或第一阶段。

![Alt text](images/2-4.png)

Embedding 将是下一章的主题。然后我们将看一下 Reranking 是如何工作的，以及它如何改进第二阶段。在本课程的最后，我们将看一下 LLM（大语言模型）如何根据之前的搜索步骤生成响应。